<b>Black-Sholes-Merton Forumla for European call Options on an underlying without dividends.</b>

 
$C(S_0,K,T,r,\sigma)=S_0N(d_1)-e^{-rT}KN(d_2)$
 
 
$N(d)=\frac{1}{\sqrt{2\pi}}\int_{-\infty}^d \mathrm{e}^{-\frac{1}{2}x^2}\mathrm{d}x$

$d_1=\frac{\log{\frac{S_0}{K}}+(r+\frac{\sigma^2}{2})T}{\sigma\sqrt{T}}$
 
$d_1=\frac{\log{\frac{S_0}{K}}+(r-\frac{\sigma^2}{2})T}{\sigma\sqrt{T}}$
 
<b>Vega Forumla.</b>

$\frac{\partial C}{\partial\sigma}=S_0N(d_1)\sqrt{T}$
 
<b>Implied Volatility Function</b>

$C(S_0,K,T,r,\sigma^2)=C^*$

<i>Resolution by Newton Scheme<i/>

$\sigma_{n+1}^{imp}=\sigma_{n}^{imp}-\frac{C(\sigma_{n}^{imp})-C^*}{\frac{\partial C(\sigma_{n}^{imp})}{\partial \sigma_{n}^{imp}}}$

In [1]:
#
# Valuation of European call options in Black-Scholes-Merton model
# incl. Vega function and implied volatility estimation
# bsm_functions.py
#
# Analytical Black-Scholes-Merton (BSM) Formula

def bsm_call_value(S0, K, T, r, sigma):
    ''' Valuation of European call option in BSM model.
    Analytical formula.
    Parameters
    ==========
    S0 : float
        initial stock/index level
    K : float
        strike price
    T : float
        maturity date (in year fractions)
    r : float
        constant risk-free short rate
    sigma : float
        volatility factor in diffusion term
    Returns
    =======
    value : float
        present value of the European call option
    '''
    from math import log, sqrt, exp
    from scipy import stats
    S0 = float(S0)
    d1 = (log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
    d2 = (log(S0 / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
    value = (S0 * stats.norm.cdf(d1, 0.0, 1.0)
            - K * exp(-r * T) * stats.norm.cdf(d2, 0.0, 1.0))
      # stats.norm.cdf --> cumulative distribution function
      #                    for normal distribution
    return value

# Vega function

def bsm_vega(S0, K, T, r, sigma):
    ''' Vega of European option in BSM model.
    Parameters
    ==========
    S0 : float
        initial stock/index level
    K : float
        strike price
    T : float
        maturity date (in year fractions)
    r : float
        constant risk-free short rate
    sigma : float
        volatility factor in diffusion term
    Returns
    =======
    vega : float
        partial derivative of BSM formula with respect
        to sigma, i.e. Vega
    '''
    from math import log, sqrt
    from scipy import stats
    S0 = float(S0)
    d1 = (log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
    vega = S0 * stats.norm.cdf(d1, 0.0, 1.0) * sqrt(T)
    return vega
          
# Implied volatility function

def bsm_call_imp_vol(S0, K, T, r, C0, sigma_est, it=100):
    ''' Implied volatility of European call option in BSM model.
    Parameters
    ==========
    S0 : float
        initial stock/index level
    K : float
        strike price
    T : float
        maturity date (in year fractions)
    r : float
        constant risk-free short rate
    sigma_est : float
        estimate of impl. volatility
    it : integer
        number of iterations
   Returns
    =======
    simga_est : float
        numerically estimated implied volatility
    '''
    for i in range(it):
        sigma_est -= ((bsm_call_value(S0, K, T, r, sigma_est) - C0)
                        / bsm_vega(S0, K, T, r, sigma_est))
    return sigma_est                

#Source: Python For Finance Yvis Hilpisch